In [4]:

import pandas as pd
import numpy as np
import  sqlalchemy as sql



Realizamos la ingesta de los datos.

In [5]:

df_amazon = pd.read_csv("../Datasets/amazon_prime_titles.csv", index_col=0 )
df_disney = pd.read_csv("../Datasets/disney_plus_titles.csv", index_col=0  )
df_hulu = pd.read_csv("../Datasets/Hulu_titles.csv", index_col=0)
df_netflix = pd.read_json("../Datasets/netflix_titles.json")

In [6]:
df_netflix.set_index("show_id", inplace=True)
df_netflix.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
show_id,,,,,,,,,,,
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


Agregamos una columna con el nombre de la plataforma.

In [7]:
df_amazon["platform"] = "amazon"
df_disney["platform"] = "disney"
df_hulu["platform"] = "hulu"
df_netflix["platform"] = "netflix"


Copiamos los valores que contienen  "min" o "Season" del dataframe de hulu y de netflix de la columna "rating" a la columna "duration", siempre y cuando esta se encuentre vacia.

In [8]:

df_hulu.loc[df_hulu["rating"].str.contains("min")
    | df_hulu["rating"].str.contains("Season")
    & df_hulu["rating"].notnull()
    & df_hulu["duration"].isnull(),"duration"] = df_hulu.loc[df_hulu["rating"].str.contains("min")
    | df_hulu["rating"].str.contains("Season")
    & df_hulu["rating"].notnull()
    & df_hulu["duration"].isnull(),"rating"]

In [9]:
df_netflix.loc[df_netflix["rating"].str.contains("min")
    | df_netflix["rating"].str.contains("Season")
    & df_netflix["rating"].notnull()
    & df_netflix["duration"].isnull(),"duration"] = df_netflix.loc[df_netflix["rating"].str.contains("min")
    | df_netflix["rating"].str.contains("Season")
    & df_netflix["rating"].notnull()
    & df_netflix["duration"].isnull(),"rating"]

Eliminamos los valores erroneos.

In [10]:

df_hulu.loc[df_hulu["rating"].str.contains("min")
    | df_hulu["rating"].str.contains("Season")
    & df_hulu["rating"].notnull()
    ,"rating"] = None

In [11]:
df_netflix.loc[df_netflix["rating"].str.contains("min")
    | df_netflix["rating"].str.contains("Season")
    & df_netflix["rating"].notnull()
    ,"rating"] = None

In [12]:
df = pd.concat([df_amazon, df_disney, df_hulu, df_netflix], ignore_index=True)

 Normalizamos columna 'duration',sepramos los datos, creamos dos nuevas columnas, dropeamos la columna sobrante y cambiamos el tipo de dato.

In [13]:


df_duracion = df["duration"].str.split(pat= " ", expand= True)

df["duration2"],df["type_duration"] =  df_duracion[0], df_duracion[1]
df.drop(["duration"],axis=1, inplace=True)


In [14]:
df.rename(columns={"duration2":"duration"},inplace=True)
df["duration"] = pd.to_numeric(df["duration"])
df.head()

,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,platform,duration,type_duration
0,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,"Comedy, Drama",A small fishing village must procure a local d...,amazon,113.0,min
1,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,amazon,110.0,min
2,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,amazon,74.0,min
3,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,Documentary,"Pink breaks the mold once again, bringing her ...",amazon,69.0,min
4,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,amazon,45.0,min


Se decide no eliminar las columnas que no se usan, porque no se conoce si puedan tener utilidad en el futuro o quienes mas podrian necesitar de las mismas. 

Por ultimo, enviamos nuestro Dataframe normalizado a nuestra base de datos.

In [16]:
engine = sql.create_engine("sqlite:///../Database/Database.db")
df.to_sql("Database", con = engine, if_exists= "replace")

22998